# Data types

In most cases, the data type (`dtype`) of a [Variable](../generated/classes/scipp.Variable.rst) is derived from the data.
For instance when passing a [numpy array](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) to scipp, scipp will use the `dtype` provided by numpy:

In [1]:
import numpy as np
import scipp as sc

var = sc.Variable(dims=['x'], values=np.arange(4.0))
var.dtype

You are running a "Debug" build of scipp. For optimal performance use a "Release" build.


DType('float64')

In [2]:
var = sc.Variable(dims=['x'], values=np.arange(4))
var.dtype

DType('int64')

The `dtype` may also be specified using a keyword argument to `sc.Variable` and most [creation functions](./creation-functions.rst#creation-functions).
It is possible to use scipp's own [scipp.DType](../generated/classes/scipp.DType.rst), [numpy.dtype](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html), or (where a numpy equivalent exists) a string:

In [3]:
var = sc.zeros(dims=['x'], shape=[2], dtype=sc.DType.float32)
var.dtype

DType('float32')

In [4]:
var = sc.zeros(dims=['x'], shape=[2], dtype=np.dtype(np.float32))
var.dtype

DType('float32')

In [5]:
var = sc.zeros(dims=['x'], shape=[2], dtype='float32')
var.dtype

DType('float32')

Scipp supports common dtypes like

- `float32`, `float64`
- `int32`, `int64`
- `bool`
- `string`
- `datetime64`

It is also possible to nest Variables, DataArrays, or Datasets inside of Variables. This is useful for storing attributes in DataArrays and Datasets. But there is only limited interoperability with numpy in those cases.

In [6]:
var = sc.scalar(sc.zeros(dims=['x'], shape=[2], dtype='float64'))
var

<scipp.Variable> ()   Variable  [dimensionless]  [<scipp.Variable> (x: 2)    float64  [dimensionless]  [0, 0]]

You can find a full list in the docs of the [scipp.DType](../generated/classes/scipp.DType.rst) class.
But note that many of those dtypes are only meant for internal use.

## Dates and Times

Scipp has a special dtype for time-points, `sc.DType.datetime64`.
Variables can be constructed using [scipp.datetime](../generated/functions/scipp.datetime.rst) and [scipp.datetimes](../generated/functions/scipp.datetimes.rst):

In [7]:
sc.datetime('2022-01-10T14:31:21')

<scipp.Variable> ()  datetime64              [s]  [2022-01-10T14:31:21]

In [8]:
sc.datetimes(dims=['t'], values=['2022-01-10T14:31:21', '2022-01-11T11:09:05'])

<scipp.Variable> (t: 2)  datetime64              [s]  [2022-01-10T14:31:21, 2022-01-11T11:09:05]

Datetimes can also be constructed from integers which encode the time since the scipp epoch which is equal to the [Unix epoch](https://en.wikipedia.org/wiki/Unix_time).
The `unit` argument determines what time scale the integers represent.

In [9]:
sc.datetime(0, unit='s')

<scipp.Variable> ()  datetime64              [s]  [1970-01-01T00:00:00]

In [10]:
sc.datetimes(dims=['t'], values=[123456789, 345678912], unit='us')

<scipp.Variable> (t: 2)  datetime64            [µs]  [1970-01-01T00:02:03.456789, 1970-01-01T00:05:45.678912]

As a shortand, [scipp.epoch](../generated/functions/scipp.epoch.rst) can be used to get a scalar containing scipp's epoch:

In [11]:
sc.epoch(unit='s')

<scipp.Variable> ()  datetime64              [s]  [1970-01-01T00:00:00]

The other creation functions also work with datetimes by specifying the `datetime64` dtype explicitly.
However, only integer inputs and Numpy arrays of [numpy.datetime64](https://numpy.org/doc/stable/reference/arrays.datetime.html#basic-datetimes) can be used in those cases.

In [12]:
sc.scalar(value=24, unit='h', dtype=sc.DType.datetime64)

<scipp.Variable> ()  datetime64              [h]  [1970-01-02T00:00:00]

In [13]:
var = sc.scalar(value=681794055, unit=sc.units.s, dtype='datetime64')
var

<scipp.Variable> ()  datetime64              [s]  [1991-08-10T03:14:15]

Scipp's datetime variables can interoperate with [numpy.datetime64](https://numpy.org/doc/stable/reference/arrays.datetime.html#basic-datetimes) and arrays thereof:

In [14]:
var.value

numpy.datetime64('1991-08-10T03:14:15')

In [15]:
sc.scalar(value=np.datetime64('now'))

<scipp.Variable> ()  datetime64              [s]  [2022-01-20T15:02:27]

Or more succinctly:

In [16]:
sc.datetime('now')

<scipp.Variable> ()  datetime64              [s]  [2022-01-20T15:02:27]

Note that `'now'` implies unit `s` even though we did not specify it.
The unit was deduced from the `numpy.datetime64` object which encodes a unit of its own.

### Operations

Variables containing datetimes only support a limited set of operations as it makes no sense to, for instance, add two time points.
In contrast to numpy, scipp does not have a separate type for time differences.
Those are simply encoded by integer Variables with a temporal unit.

In [17]:
a = sc.datetime('2021-03-14T00:00:00', unit='ms')
b = sc.datetime('2000-01-01T00:00:00', unit='ms')
a - b

<scipp.Variable> ()      int64             [ms]  [668995200000]

In [18]:
a + b

DTypeError: 'add' does not support dtypes 'datetime64', 'datetime64', 

In [19]:
a + sc.scalar(value=123, unit='ms')

<scipp.Variable> ()  datetime64             [ms]  [2021-03-14T00:00:00.123]

### Time zones

Scipp does not support manual handling of time zones.
All datetime objects are assumed to be in UTC.
Scipp does not look at your local time zone, thus the following will always produce 12:30 on 2021-03-09 UTC no matter where you are when you run this code:

In [20]:
sc.scalar(value=np.datetime64('2021-09-03T12:30:00'))

<scipp.Variable> ()  datetime64              [s]  [2021-09-03T12:30:00]